In [7]:
import pandas as pd
import numpy as np
# operation for rearrangin tabular data

In [4]:
# called reshaping or pivot operation

In [5]:
# stack: columns to rows
# unstack: rows to columns

In [8]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
index=pd.Index(['Ohio', 'Colorado'], name='state'),
columns=pd.Index(['one', 'two', 'three'],
name='number'))

In [9]:
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [11]:
result = data.stack()

In [13]:
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [15]:
result.index.names

FrozenList(['state', 'number'])

In [18]:
result1 = result.unstack() 

In [17]:
# innermost goes to column

In [19]:
result1

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [22]:
result1.unstack(0)

number  state   
one     Ohio        0
        Colorado    3
two     Ohio        1
        Colorado    4
three   Ohio        2
        Colorado    5
dtype: int64

In [21]:
result1.stack()

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [23]:
# unstacking() might produce missing values
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])

In [24]:
s1

a    0
b    1
c    2
d    3
dtype: int64

In [25]:
s2

c    4
d    5
e    6
dtype: int64

In [28]:
data2 = pd.concat([s1,s2],keys=['one','two'])

In [29]:
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [30]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [31]:
data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

In [32]:
# when you unstack a dataframe, the level unstacked becomes the lowest level in the
# result

In [33]:
df = pd.DataFrame({'left': result, 'right': result + 5},
columns=pd.Index(['left', 'right'], name='side'))

In [34]:
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [35]:
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [36]:
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [37]:
df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [38]:
df.unstack('state').stack('side')

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

#### Pivoting 

In [39]:
data = pd.read_csv('Data/macrodata.csv')

In [40]:
data

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.980,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.150,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.350,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.370,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.540,139.6,3.50,5.2,180.007,2.31,1.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,2008.0,3.0,13324.600,9267.7,1990.693,991.551,9838.3,216.889,1474.7,1.17,6.0,305.270,-3.16,4.33
199,2008.0,4.0,13141.920,9195.3,1857.661,1007.273,9920.4,212.174,1576.5,0.12,6.9,305.952,-8.79,8.91
200,2009.0,1.0,12925.410,9209.2,1558.494,996.287,9926.4,212.671,1592.8,0.22,8.1,306.547,0.94,-0.71
201,2009.0,2.0,12901.504,9189.0,1456.678,1023.528,10077.5,214.469,1653.6,0.18,9.2,307.226,3.37,-3.19


In [41]:
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter,name='date')

In [42]:
periods

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name='date', length=203)

In [43]:
columns = pd.Index(['realgdp','infl','unemp'],name='item')

In [44]:
columns

Index(['realgdp', 'infl', 'unemp'], dtype='object', name='item')

In [45]:
data = data.reindex(columns=columns)

In [46]:
data 
# setting column index

item,realgdp,infl,unemp
0,2710.349,0.00,5.8
1,2778.801,2.34,5.1
2,2775.488,2.74,5.3
3,2785.204,0.27,5.6
4,2847.699,2.31,5.2
...,...,...,...
198,13324.600,-3.16,6.0
199,13141.920,-8.79,6.9
200,12925.410,0.94,8.1
201,12901.504,3.37,9.2


In [47]:
data.index = periods.to_timestamp('D','end')

In [48]:
data

item,realgdp,infl,unemp
date,,,
1959-03-31 23:59:59.999999999,2710.349,0.00,5.8
1959-06-30 23:59:59.999999999,2778.801,2.34,5.1
1959-09-30 23:59:59.999999999,2775.488,2.74,5.3
1959-12-31 23:59:59.999999999,2785.204,0.27,5.6
1960-03-31 23:59:59.999999999,2847.699,2.31,5.2
...,...,...,...
2008-09-30 23:59:59.999999999,13324.600,-3.16,6.0
2008-12-31 23:59:59.999999999,13141.920,-8.79,6.9
2009-03-31 23:59:59.999999999,12925.410,0.94,8.1


In [54]:
ldata = data.stack().reset_index().rename(columns={0:'value'})

In [55]:
ldata

,date,item,value
0,1959-03-31 23:59:59.999999999,realgdp,2710.349
1,1959-03-31 23:59:59.999999999,infl,0.000
2,1959-03-31 23:59:59.999999999,unemp,5.800
3,1959-06-30 23:59:59.999999999,realgdp,2778.801
4,1959-06-30 23:59:59.999999999,infl,2.340
...,...,...,...
604,2009-06-30 23:59:59.999999999,infl,3.370
605,2009-06-30 23:59:59.999999999,unemp,9.200
606,2009-09-30 23:59:59.999999999,realgdp,12990.341
607,2009-09-30 23:59:59.999999999,infl,3.560


In [56]:
# PIVOT
# The first two values passed are the columns to be used respectively as the row and
# column index, then finally an optional value column to fill the DataFrame.

In [57]:
# date: row index, intem: column index
pivoted = ldata.pivot('date','item','value')

In [58]:
pivoted

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2
...,...,...,...
2008-09-30 23:59:59.999999999,-3.16,13324.600,6.0
2008-12-31 23:59:59.999999999,-8.79,13141.920,6.9
2009-03-31 23:59:59.999999999,0.94,12925.410,8.1


In [62]:
# Suppose
# you had two value columns that you wanted to reshape simultaneously:
len(ldata)

609

In [60]:
ldata['value2'] = np.random.randn(len(ldata))

In [61]:
ldata

,date,item,value,value2
0,1959-03-31 23:59:59.999999999,realgdp,2710.349,0.205462
1,1959-03-31 23:59:59.999999999,infl,0.000,-0.083845
2,1959-03-31 23:59:59.999999999,unemp,5.800,-1.858993
3,1959-06-30 23:59:59.999999999,realgdp,2778.801,-1.221577
4,1959-06-30 23:59:59.999999999,infl,2.340,0.985555
...,...,...,...,...
604,2009-06-30 23:59:59.999999999,infl,3.370,-0.381840
605,2009-06-30 23:59:59.999999999,unemp,9.200,-0.225211
606,2009-09-30 23:59:59.999999999,realgdp,12990.341,1.906422
607,2009-09-30 23:59:59.999999999,infl,3.560,-0.386798


In [63]:
# if you omit the last argument, you obtain a DF with hierarchial indexing

In [64]:
pivoted = ldata.pivot('date','item')

In [65]:
pivoted

value                     value2            \
item                           infl    realgdp unemp      infl   realgdp   
date                                                                       
1959-03-31 23:59:59.999999999  0.00   2710.349   5.8 -0.083845  0.205462   
1959-06-30 23:59:59.999999999  2.34   2778.801   5.1  0.985555 -1.221577   
1959-09-30 23:59:59.999999999  2.74   2775.488   5.3 -0.641577 -0.358168   
1959-12-31 23:59:59.999999999  0.27   2785.204   5.6  0.910717  2.307847   
1960-03-31 23:59:59.999999999  2.31   2847.699   5.2  0.029123 -0.488247   
...                             ...        ...   ...       ...       ...   
2008-09-30 23:59:59.999999999 -3.16  13324.600   6.0  1.547354  0.578114   
2008-12-31 23:59:59.999999999 -8.79  13141.920   6.9  1.361326  0.497604   
2009-03-31 23:59:59.999999999  0.94  12925.410   8.1  0.542933  0.663010   
2009-06-30 23:59:59.999999999  3.37  12901.504   9.2 -0.381840 -0.388281   
2009-09-30 23:59:59.999999999  3.56  12990.341   9.6 -0.386798  1.906422   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999 -1.858993  
1959-06-30 23:59:59.999999999 -0.891592  
1959-09-30 23:59:59.999999999 -0.968651  
1959-12-31 23:59:59.999999999 -0.488280  
1960-03-31 23:59:59.999999999  0.373795  
...                                 ...  
2008-09-30 23:59:59.999999999 -2.916922  
2008-12-31 23:59:59.999999999  1.057151  
2009-03-31 23:59:59.999999999 -0.991780  
2009-06-30 23:59:59.999999999 -0.225211  
2009-09-30 23:59:59.999999999  0.710529  

[203 rows x 6 columns]

In [66]:
pivoted['value'][:5]

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2


In [67]:
# Pivoting:
# wide to long format

In [68]:
# for long to wide: you use pivot

In [69]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
'A': [1, 2, 3],
'B': [4, 5, 6],
'C': [7, 8, 9]})

In [70]:
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [71]:
# .melt: melts multiple columns

In [72]:
# first find the group indicator
melted = pd.melt(df,['key'])

In [73]:
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [74]:
# reshaped using pivot
reshaped = melted.pivot('key','variable','value')

In [78]:
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


In [79]:
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [80]:
# specify a subset of columns to be used as a value columns:
pd.melt(df,id_vars=['key'],value_vars=['A','B'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
